In [ ]:
! pip install youtube-dl
! pip install git+https://github.com/Cupcakus/pafy

In [ ]:
import torch
import numpy as np
import cv2
import pafy
import time

In [ ]:
def Get_From_Image(url):
  openApiURL = "http://aiopen.etri.re.kr:8000/ObjectDetect"
  accessKey = "493be777-1dc2-4b92-90a7-4e25b97c945f"
  play = pafy.new(url).streams[-1]
  assert play is not None
  player = cv2.VideoCapture(play.url)
  assert player.isOpened()
  frame_count=player.get(cv2.CAP_PROP_FRAME_COUNT)
  frame_count=int(frame_count/20)
  print(frame_count)
  count = 0
  result = []
  while  True:
      ret, frame = player.read()
      if ret != True:
          break
      
      if(int(player.get(1)) % frame_count == 0) :
          # 이미지 받는 경로
          cv2.imwrite( "C:/Users/kimseyun/Desktop/etri/image%d.jpg" % count, frame,[cv2.IMWRITE_JPEG_QUALITY, 80])
          count += 1

  result_data=[]
  temp = 0
  while temp<count:
      # 이미지 여는 경로
      image = open('C:/Users/kimseyun/Desktop/etri/image%d.jpg' %temp , "rb")
      image_content = base64.b64encode(image.read()).decode("utf8")
      requestJson = {
          "access_key": accessKey,
          "argument": {
              "type": 'jpg',
              "file": image_content
          }
      }
      http = urllib3.PoolManager()
      response = http.request(
          "POST",
          openApiURL,
          headers={"Content-Type": "application/json; charset=UTF-8"},
          body=json.dumps(requestJson)
      )
      temp += 1
      result = json.loads(response.data)
      if(result['result']!=-1):
          if( 'data' in result['return_object']):
              for i in range(len(result['return_object']['data'])):
                  result_data.append(result['return_object']['data'][i]['class'])

  # 빈도 순으로 정렬
  result_data=sorted(result_data, key=lambda x: (-result_data.count(x), result_data.index(x)))
  # 중복 제거
  result_data = reduce(lambda acc, cur: acc if cur in acc else acc+[cur], result_data, [])
  return result_data

In [ ]:
# 파파고 번역
def get_translate(text):
    client_id = "6rBCYe4g5L7a6u38mcn5" # <-- client_id 기입
    client_secret = "qzLdHeUJ0O" # <-- client_secret 기입

    data = {'text' : text,
            'source' : 'en',
            'target': 'ko'}

    url = "https://openapi.naver.com/v1/papago/n2mt"

    header = {"X-Naver-Client-Id":client_id,
              "X-Naver-Client-Secret":client_secret}

    response = requests.post(url, headers=header, data=data)
    rescode = response.status_code

    if(rescode==200):
        send_data = response.json()
        trans_data = (send_data['message']['result']['translatedText'])
        return trans_data
    else:
        print("Error Code:" , rescode)

In [ ]:
def listToString(str_list):
    result = ""
    for s in str_list:
        result += s + " "
    return result.strip()

In [ ]:
def Check(url):
  All_list = []
  Video = ObjectDetection(url, All_list)
  Video()
  # 빈도 순으로 정렬
  result = sorted(All_list, key=lambda x: (-All_list.count(x), All_list.index(x)))

  # 중복 제거
  result = reduce(lambda acc, cur: acc if cur in acc else acc+[cur], result, [])
  
  one=result[:11]
  one = get_translate(one)
  if '사람' in one:
    one.remove('사람')
  print(listToString(one))
  two=one[:5]
  print(listToString(two))

In [ ]:
Check("https://www.youtube.com/watch?v=T8yJ52L3kH0")